<a href="https://colab.research.google.com/github/chaiyixuan/GCN/blob/master/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1.切换TF为1.5版本
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

#2.切换工作路径到当前项目
import os
 
print(os.getcwd())#查看工作路径
 
# 修改工作路径
import os

from google.colab import drive
drive.mount('/content/drive')
 
path = "/content/drive/My Drive/SpellGCN"
os.chdir(path)

TensorFlow 1.x selected.
/content
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
!nvidia-smi


Wed Aug 12 11:09:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    55W / 149W |     69MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.  # You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Run masked LM/next sentence masked_lm pre-training for BERT."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import os, time
import modeling
import optimization
import tensorflow as tf

import numpy, time
import collections
import csv
import os
import modeling
import optimization
import tokenization
import tensorflow as tf
from tensorflow.python.estimator import training
from tensorflow.core.protobuf import rewriter_config_pb2
import sys

#os.environ["CUDA_VISIBLE_DEVICES"]="0"
#reload(sys)
#sys.setdefaultencoding("utf8")

flags = tf.flags

FLAGS = flags.FLAGS

## Required parameters
flags.DEFINE_string(
    "bert_config_file", "model/bert_config.json",
    "The config json file corresponding to the pre-trained BERT model. "
    "This specifies the model architecture.")

flags.DEFINE_string(
    "input_file", None,
    "Input TF example files (can be a glob or comma separated).")

flags.DEFINE_string(
    "data_dir", "data/merged/",
    #"data_dir", "data/sighan15/",
    "The input data dir. Should contain the .tsv files (or other data files) "
    "for the task.")

flags.DEFINE_string(
    "graph_dir", "data/gcn_graph.ty_xj",
    "The dir for graph data")

flags.DEFINE_string(
    "eval_file", None,
    "TF example files for eval (can be a glob or comma separated).")

flags.DEFINE_string(
    "output_dir", "output2/",
    "The output directory where the model checkpoints will be written.")

flags.DEFINE_string("task_name", "CSC", "The name of the task to train.")
flags.DEFINE_string("job_name", "SpellGCN", "The name of the job to train.")
flags.DEFINE_string("vocab_file", "model/vocab.txt",
                    "The vocabulary file that the BERT model was trained on.")

flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")

flags.DEFINE_bool(
    "continual", False,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")
## Other parameters
flags.DEFINE_string(
    "init_checkpoint", "model/bert_model.ckpt",
    "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_integer(
    "max_seq_length",128,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded. Must match data generation.")

flags.DEFINE_integer(
    "max_predictions_per_seq",128,
    "Maximum number of masked LM predictions per sequence. "
    "Must match data generation.")
flags.DEFINE_integer(
    "max_nodes", 4755,
    "Maximum number of adjacent nodes. "
    "Must match data generation.")

flags.DEFINE_integer(
    "max_relations", 2,
    "Maximum number relations.")
flags.DEFINE_integer("next_sent_type", 3, "Next sentence task: support 2 or 3 only. Must match data generation.")

flags.DEFINE_bool("do_train",True, "Whether to run training.")

flags.DEFINE_bool("do_eval", False, "Whether to run eval on the dev set.")

flags.DEFINE_bool("do_predict", False, "Whether to run eval on the test set.")

flags.DEFINE_bool("with_error",False, "Whether to run eval on the test set.")

flags.DEFINE_integer("train_batch_size", 32, "Total batch size for training.")

flags.DEFINE_integer("eval_batch_size", 1, "Total batch size for eval.")

#flags.DEFINE_float("learning_rate", 2e-5, "The initial learning rate for Adam.")
flags.DEFINE_float("learning_rate", 1.40e-5, "The initial learning rate for Adam.")
#flags.DEFINE_float("learning_rate", 1.40e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 10.0,
                   "Total number of training epochs to perform.")

flags.DEFINE_integer("num_warmup_steps", 10000, "Number of warmup steps.")



flags.DEFINE_integer("save_checkpoints_steps", 10000,
                     "How often to save the model checkpoint.")

flags.DEFINE_integer("keep_checkpoint_max",1,
                     "Maximum number of checkpoints to keep.")

flags.DEFINE_integer("iterations_per_loop", 5000,
                     "How many steps to make in each estimator call.")

flags.DEFINE_integer("max_eval_steps", 10000, "Maximum number of eval steps.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")

tf.flags.DEFINE_string(
    "tpu_name", None,
    "The Cloud TPU to use for training. This should be either the name "
    "used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 "
    "url.")

tf.flags.DEFINE_string(
    "tpu_zone", None,
    "[Optional] GCE zone where the Cloud TPU is located in. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

tf.flags.DEFINE_string(
    "gcp_project", None,
    "[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

tf.flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

flags.DEFINE_integer(
    "num_tpu_cores", 1,
    "Only used if `use_tpu` is True. Total number of TPU cores to use.")

flags.DEFINE_bool("use_horovod", False, "Whether to use Horovod for multi-gpu training.")

flags.DEFINE_integer("random_seed", None, "Random seed for data generation.")

class LossHook(tf.train.SessionRunHook):
    def __init__(self, intervel):
        self.intervel = intervel
        self.step = 0
        self.fetches = tf.train.SessionRunArgs(
          fetches=[
          "global_step:0",
          "total_loss:0",
          "masked_lm_loss:0",
          "learning_rate:0"
        ])
        self.step_start_time = -1

    def before_run(self, run_context):
      self.step_start_time = time.time()
      return self.fetches

    def after_run(self, run_context, run_values):
        if (not FLAGS.use_horovod) and self.step % self.intervel == 0:
            global_step, total_loss, masked_lm_loss, learning_rate = run_values.results
            tf.logging.info('global_step=%d\ttotal_loss=%2.6f\tmasked_lm_loss=%2.6f\tlearning_rate=%.6e' % (
                global_step, total_loss, masked_lm_loss, learning_rate))
        self.step += 1


class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, masked_lm_positions, masked_lm_ids, masked_lm_weights=None, text_b=None, label=None):
    """Constructs a InputExample.

    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.masked_lm_positions = masked_lm_positions
    self.masked_lm_ids = masked_lm_ids
    self.masked_lm_weights = masked_lm_weights
    self.text_b = text_b
    self.label = label

class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.

  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.

  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               masked_lm_positions,
               masked_lm_ids,
               masked_lm_weights,
               label_id,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.masked_lm_positions = masked_lm_positions
    self.masked_lm_ids = masked_lm_ids
    self.masked_lm_weights = masked_lm_weights
    self.label_id = label_id
    self.is_real_example = is_real_example


# TODO: L2
def gcnLayer(gcn_in, in_dim, gcn_dim, batch_size, max_nodes, 
	max_labels, adj_mat, w_gating=True, num_layers=1, dropout=1.0, name="GCN"):
  out = []
  out.append(gcn_in)
   
  for layer in range(num_layers):
    gcn_in = out[-1]
    if len(out) > 1: in_dim = gcn_dim

    with tf.name_scope("%s-%d" %(name, layer)):
      #with tf.variable_scope("Loop-name-%s-layer-%d" %(name, layer)) as scope:
      #  w_loop = tf.get_variable("w_loop", initializer=tf.eye(in_dim), trainable=False)
      #  inp_loop = tf.tensordot(gcn_in, w_loop, axes=[1,0])
      #  if dropout != 1.0: inp_loop = tf.nn.dropout(inp_loop, keep_probs=dropout)
      #  loop_act = inp_loop
      act_sum = out[-1]
      _act_sum = tf.zeros_like(gcn_in)
      #else:
      #  act_sum = tf.zeros_like(gcn_in)
      #_act_sum = tf.zeros_like(gcn_in)
      for lbl in range(FLAGS.max_relations):
         with tf.variable_scope("label-%d-name-%s_layer-%d" %(lbl, name, layer), reuse=tf.AUTO_REUSE) as scope:
           w_in = tf.get_variable('w_in', initializer=tf.eye(gcn_dim), trainable=True, )
           b_in = tf.get_variable('b_in', [1, gcn_dim], trainable=True, initializer=tf.zeros_initializer())

           if w_gating:
             w_gin = tf.get_variable('w_gin', [gcn_dim, 1], initializer=tf.contrib.layers.xavier_initializer())
             b_gin = tf.get_variable('b_gin', [1], initializer=tf.zeros_initializer())
        
         with tf.name_scope('arcs-%d_name-%s_layer-%d' %(lbl, name, layer)):
           inp_in = tf.matmul(gcn_in, w_in) + b_in # [n, d]
           if adj_mat is not None:
             adj_matrix = adj_mat[lbl]
             in_t = tf.matmul(tf.cast(adj_matrix, tf.float32), inp_in) #[n, d]
           else:
             in_t = inp_in
           
           if dropout != 1.0: in_t = tf.nn.dropout(in_t, keep_prob=dropout)
           if w_gating and adj_mat is not None:
             inp_gin = tf.sigmoid(tf.matmul(gcn_in, w_gin) + tf.to_float(b_gin[None,:])) #[n, 1]
             in_act = tf.matmul(tf.cast(adj_matrix, tf.float32), inp_gin * inp_in) #[n, d]
           else:
             in_act = in_t
         if lbl == 0:
           multi_graph_embs = in_act[None,:,:]
         else:
           multi_graph_embs = tf.concat([multi_graph_embs, in_act[None,:,:]], axis=0) #[MAX_REL, n, d]
      
      with tf.variable_scope("relation_prototype_scope", reuse=tf.AUTO_REUSE):
        w0 = tf.get_variable(
          "relation_prototype",
          shape=[2,in_dim, 1],
          initializer=tf.contrib.layers.xavier_initializer())
        graph_weights = tf.matmul(multi_graph_embs, w0) / 3
        graph_weights = tf.nn.softmax(graph_weights, axis=0)
        _act_sum = tf.reduce_sum(graph_weights * multi_graph_embs, axis=0) 

      act_sum += _act_sum
      if layer != 0:
        act_sum += out[0]

      #gcn_out = tf.nn.relu(act_sum) if layer != num_layers -1 else act_sum
      gcn_out = act_sum
      out.append(gcn_out)
  return out[-1]
             

def build_gcn_output(adj_mat, w2n, n2w, embedding_table, bert_config, is_training):
    for lbl in range(FLAGS.max_relations):
      adj = adj_mat[lbl] 
      #adj = adj / (adj_mat.sum(0, keepdims=True) + 1e-6)
      adj = adj + np.identity(FLAGS.max_nodes, np.int)
      adj_mat[lbl] = adj

    _adj_mat = tf.to_float(tf.convert_to_tensor(adj_mat)) 
    D = tf.pow(tf.reduce_sum(_adj_mat, axis=2) + 1e-8, -0.5)
    D = tf.matrix_diag(D)
    _adj_mat = tf.keras.backend.batch_dot(_adj_mat, D)
    #print(_adj_mat)
    _adj_mat = tf.transpose(_adj_mat, [0,2,1])
    _adj_mat = tf.keras.backend.batch_dot(_adj_mat,D)

    _w2n = tf.to_int32(tf.convert_to_tensor(w2n))
    _n2w = tf.to_int32(tf.convert_to_tensor(n2w))
    with tf.variable_scope("gcn"):
      node_embedding = gcnLayer(
	  gcn_in = tf.gather(embedding_table, _n2w), #n2w'size is FLAGS.nodes, max value is 21127
	  in_dim=bert_config.hidden_size, 
	  gcn_dim=bert_config.hidden_size, 
          batch_size=32,
          max_nodes=FLAGS.max_nodes,
          max_labels=FLAGS.max_relations,
	  adj_mat=_adj_mat,
          w_gating=False,
          num_layers=2,
          dropout = 0.9 if is_training else 1.0,
          name="GCN") #return [FLAGS.max_nodes, hidden_size]
    
    rest_embedding = embedding_table
    """
    with tf.variable_scope("gcn", reuse=True):
      rest_embedding = gcnLayer(
	  gcn_in = model.get_embedding_table(), #n2w'size is FLAGS.nodes, max value is 21127
	  in_dim=bert_config.hidden_size, 
	  gcn_dim=bert_config.hidden_size, 
          batch_size=32,
          max_nodes=FLAGS.max_nodes,
          max_labels=FLAGS.max_relations,
	  adj_mat=None,
          w_gating=False,
          num_layers=1,
          dropout = 1.0 if is_training else 1.0,
          name="GCN") #return [FLAGS.max_nodes, hidden_size]
    """
    expanded_node_embedding = tf.gather(node_embedding, _w2n) #w2n's is is 21127, max value is FLAGS.nodes
    mask_nodes_ids = tf.to_float(tf.not_equal(_w2n,0))
    gcn_embedding = mask_nodes_ids[:,None] * expanded_node_embedding + (1 - mask_nodes_ids[:,None]) *  rest_embedding

    return gcn_embedding


def model_fn_builder(adj_mat, w2n, n2w, bert_config, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    masked_lm_positions = features["masked_lm_positions"]
    masked_lm_ids = features["masked_lm_ids"]
    masked_lm_weights = features["masked_lm_weights"]
    #next_sentence_labels = features["next_sentence_labels"]
   
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)
    
    gcn_embedding = build_gcn_output(adj_mat, w2n, n2w, model.get_embedding_table(), bert_config, is_training)
 
    (masked_lm_loss,
     masked_lm_example_loss, masked_lm_log_probs) = get_masked_lm_output(
         bert_config, model.get_sequence_output(), gcn_embedding,
         masked_lm_positions, masked_lm_ids, masked_lm_weights)


    masked_lm_loss = tf.identity(masked_lm_loss, name="masked_lm_loss")


    total_loss = masked_lm_loss

    total_loss = tf.identity(total_loss, name='total_loss')

    tvars = tf.trainable_variables()

    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint and (not FLAGS.use_horovod ):
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    if not FLAGS.use_horovod :
      tf.logging.info("**** Trainable Variables ****")
      for var in tvars:
        init_string = ""
        if var.name in initialized_variable_names:
          init_string = ", *INIT_FROM_CKPT*"
        tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                        init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu, FLAGS.use_horovod)

      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op)
      return output_spec
    elif mode == tf.estimator.ModeKeys.PREDICT:

      #def metric_fn(masked_lm_example_loss, masked_lm_log_probs, masked_lm_ids,
      #              masked_lm_weights):#, next_sentence_example_loss,
                    #next_sentence_log_probs, next_sentence_labels):
      """Computes the loss and accuracy of the model."""
      #masked_lm_log_probs = tf.reshape(masked_lm_log_probs,
      #                                   [-1, masked_lm_log_probs.shape[-1]])
      masked_lm_predictions = tf.argmax(
         masked_lm_log_probs, axis=-1, output_type=tf.int32)
        #    values=next_sentence_example_loss)

      predictions = {
          "input_ids": tf.reshape(input_ids, [-1]),
          "predictions": masked_lm_log_probs
      }

      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          predictions=predictions)
          #eval_metric_ops=eval_metrics)
      return output_spec
    else:
      def metric_fn(masked_lm_example_loss, masked_lm_log_probs, masked_lm_ids,
                    masked_lm_weights):
        """Computes the loss and accuracy of the model."""
        masked_lm_log_probs = tf.reshape(masked_lm_log_probs,
                                         [-1, masked_lm_log_probs.shape[-1]])
        masked_lm_predictions = tf.argmax(
            masked_lm_log_probs, axis=-1, output_type=tf.int32)
        masked_lm_example_loss = tf.reshape(masked_lm_example_loss, [-1])
        masked_lm_ids = tf.reshape(masked_lm_ids, [-1])
        masked_lm_weights = tf.reshape(masked_lm_weights, [-1])
        masked_lm_accuracy = tf.metrics.accuracy(
            labels=masked_lm_ids,
            predictions=masked_lm_predictions,
            weights=masked_lm_weights)
        masked_lm_mean_loss = tf.metrics.mean(
            values=masked_lm_example_loss, weights=masked_lm_weights)

        return {
            "masked_lm_accuracy": masked_lm_accuracy,
            "masked_lm_loss": masked_lm_mean_loss,
        }

      eval_metrics = metric_fn(
          masked_lm_example_loss, masked_lm_log_probs, masked_lm_ids,
          masked_lm_weights)
      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metric_ops=eval_metrics)

      return output_spec

  return model_fn



def get_masked_lm_output(bert_config, input_tensor, output_weights, positions,
                         label_ids, label_weights):
  """Get loss and log probs for the masked LM."""
  input_tensor = gather_indexes(input_tensor, positions)

  with tf.variable_scope("cls/predictions"):
    # We apply one more non-linear transformation before the output layer.
    # This matrix is not used after pre-training.
    with tf.variable_scope("transform"):
      input_tensor = tf.layers.dense(
          input_tensor,
          units=bert_config.hidden_size,
          activation=modeling.get_activation(bert_config.hidden_act),
          kernel_initializer=modeling.create_initializer(
              bert_config.initializer_range))
      input_tensor = modeling.layer_norm(input_tensor)

    # The output weights are the same as the input embeddings, but there is
    # an output-only bias for each token.
    output_bias = tf.get_variable(
        "output_bias",
        shape=[bert_config.vocab_size],
        initializer=tf.zeros_initializer())
    logits = tf.matmul(input_tensor, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    label_ids = tf.reshape(label_ids, [-1])
    label_weights = tf.reshape(label_weights, [-1])

    one_hot_labels = tf.one_hot(
        label_ids, depth=bert_config.vocab_size, dtype=tf.float32)

    # The `positions` tensor might be zero-padded (if the sequence is too
    # short to have the maximum number of predictions). The `label_weights`
    # tensor has a value of 1.0 for every real prediction and 0.0 for the
    # padding predictions.
    per_example_loss = -tf.reduce_sum(log_probs * one_hot_labels, axis=[-1])
    numerator = tf.reduce_sum(label_weights * per_example_loss)
    denominator = tf.reduce_sum(label_weights) + 1e-5
    loss = numerator / denominator

  return (loss, per_example_loss, log_probs)


def gather_indexes(sequence_tensor, positions):
  """Gathers the vectors at the specific positions over a minibatch."""
  sequence_shape = modeling.get_shape_list(sequence_tensor, expected_rank=3)
  batch_size = sequence_shape[0]
  seq_length = sequence_shape[1]
  width = sequence_shape[2]

  flat_offsets = tf.reshape(
      tf.range(0, batch_size, dtype=tf.int32) * seq_length, [-1, 1])
  flat_positions = tf.reshape(positions + flat_offsets, [-1])
  flat_sequence_tensor = tf.reshape(sequence_tensor,
                                    [batch_size * seq_length, width])
  output_tensor = tf.gather(flat_sequence_tensor, flat_positions)
  return output_tensor


class DataProcessor(object):
  """Base class for data converters for sequence classification data sets."""
  def get_train_full_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the train set."""
    raise NotImplementedError()


  def get_mode(self):
    return "classification"
  def get_labels(self):
    """Gets the list of labels for this data set."""
    raise NotImplementedError()

  @classmethod
  def _read_tsv(cls, input_file, quotechar=None):
    """Reads a tab separated value file."""
    with tf.gfile.Open(input_file, "r") as f:
      reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
      lines = []
      for line in reader:
        lines.append(line)
      return lines


class CSCProcessor(DataProcessor):
  """Processor for the XNLI data set."""

  def __init__(self):
    self.language = "zh"
    
  def get_dev_full_examples(self, data_dir):
    """See base class."""
    if FLAGS.with_error:
      input_file = os.path.join(data_dir, "TestInputWithError.txt") 
      truth_file = os.path.join(data_dir, "TestTruthWithError.txt") 
    else:
      input_file = os.path.join(data_dir, "TestInput.txt") 
      truth_file = os.path.join(data_dir, "TestTruth.txt") 
     
    #input_file = os.path.join(data_dir, "Training/B1_training_input.txt") 
    #truth_file = os.path.join(data_dir, "Training/B1_training_truth.txt") 
    examples = []
    for i,(inp,trth) in enumerate(
	zip(open(input_file,encoding="utf-8"), open(truth_file,encoding="utf-8"))):
      inp_fields = inp.strip().split("\t")
      trth_fields = trth.strip().split(" ")
      # Only tackle the sentence withError.
      #if len(trth_fields) == 2:
      #  continue 
      guid = inp_fields[0]  
      text_a = tokenization.convert_to_unicode(inp_fields[1])
      text_a = text_a[:FLAGS.max_seq_length-2]
      masked_lm_positions = [j + 1 for j in range(len(text_a))]
      masked_lm_tokens = [t for t in text_a]
      masked_lm_weights = [1.0 for t in text_a]

      text_b = None
      label = "0"
      for j, pos_tok in enumerate(trth_fields[1:]):
        if j % 2 == 0: 
          pos = pos_tok.strip(",")
        else:
          # replace mispell tokens of the input sentence to correct tokens.
          if int(pos) < FLAGS.max_seq_length and int(pos) <= len(masked_lm_tokens):
            masked_lm_tokens[int(pos)-1] = pos_tok.strip(",")
      examples.append(
          InputExample(guid=guid, 
			text_a=text_a, 
			masked_lm_positions=masked_lm_positions, 
			masked_lm_ids=masked_lm_tokens, 
			masked_lm_weights=masked_lm_weights,
			text_b=text_b,
			label=label))
    return examples

  def get_train_full_examples(self, data_dir):
    """See base class."""
    if not FLAGS.continual:
      input_file = os.path.join(data_dir, "TrainingInputAll.txt") 
      truth_file = os.path.join(data_dir, "TrainingTruthAll.txt") 
    else:
      input_file = os.path.join(data_dir, "TrainingInputWithError.txt") 
      truth_file = os.path.join(data_dir, "TrainingTruthWithError.txt")
    
    examples = []
    for i,(inp,trth) in enumerate(
	zip(open(input_file,encoding="utf-8"), open(truth_file,encoding="utf-8"))):
      inp_fields = inp.strip().split("\t")
      trth_fields = trth.strip().split(" ")
     
      guid = inp_fields[0]  
      text_a = tokenization.convert_to_unicode(inp_fields[1])
      text_a = text_a[:FLAGS.max_seq_length-2]
      masked_lm_positions = [j + 1 for j in range(len(text_a))]
      masked_lm_tokens = [t for t in text_a]
      masked_lm_weights = [1.0 for t in text_a]
      text_b = None
      label = "0"
      for j, pos_tok in enumerate(trth_fields[1:]):
        if j % 2 == 0: 
          pos = pos_tok.strip(",")
        else:
        # replace mispell tokens of the input sentence to correct tokens.
          if int(pos) < FLAGS.max_seq_length -2:
            masked_lm_tokens[int(pos)-1] = pos_tok.strip(",")
      examples.append(
          InputExample(guid=guid, 
			text_a=text_a, 
			masked_lm_positions=masked_lm_positions, 
			masked_lm_ids=masked_lm_tokens, 
			masked_lm_weights=masked_lm_weights,
			text_b=text_b,
			label=label))
    return examples

  def get_labels(self):
    """See base class."""
    return ["0", "1"]

def convert_single_example(ex_index, example, label_list, output_mode, max_seq_length, max_predictions_per_seq,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""

  if isinstance(example, PaddingInputExample):
    return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=0,
        is_real_example=False)

  label_map = {}
  for (i, label) in enumerate(label_list):
    label_map[label] = i
  #BERT will add # at head of the number
  #tokens_a = tokenizer.tokenize(example.text_a) 
  tokens_a = [a for a in example.text_a]
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  tokens = []
  segment_ids = []
  tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
  tokens.append("[SEP]")
  segment_ids.append(0)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)
  
  
  for j, tok in enumerate(tokens):
    if tok not in tokenizer.vocab:
      tokens[j] = "[UNK]"
       
  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)
  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length
  
  #masked_lm_ids = tokenizer.tokenize("".join(example.masked_lm_ids).decode("utf8"))
  masked_lm_ids = []
  masked_lm_ids.append("[CLS]")
  for tok in example.masked_lm_ids:
     masked_lm_ids.append(tok if tok in tokenizer.vocab else "[UNK]")
  masked_lm_ids.append("[SEP]")
  masked_lm_ids = tokenizer.convert_tokens_to_ids(masked_lm_ids)  
     

  masked_lm_weights = [1.0] * len(masked_lm_ids)
  masked_lm_weights[0] = 0.0
  masked_lm_weights[len(masked_lm_ids)-1] = 0.0
  masked_lm_positions = [i for i in range(len(masked_lm_ids))]

  while len(masked_lm_positions) < max_predictions_per_seq:
    masked_lm_positions.append(0)
    masked_lm_ids.append(0)
    masked_lm_weights.append(0.0)

  assert len(masked_lm_ids) == max_predictions_per_seq
  assert len(masked_lm_positions) == max_predictions_per_seq
  assert len(masked_lm_weights) == max_predictions_per_seq

  if output_mode == 'classification':
    label_id = label_map[example.label]
  elif output_mode == 'regression':
    label_id = float(example.label)

  else:
    raise KeyError(mode)
  if ex_index < 20:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.logging.info("masked_lm_positions: %s" % " ".join([str(x) for x in masked_lm_positions]))
    tf.logging.info("masked_lm_ids: %s" % " ".join([str(x) for x in masked_lm_ids]))
    tf.logging.info("masked_lm_tokens: %s" % " ".join(
	[str(x) for x in tokenizer.convert_ids_to_tokens(masked_lm_ids)]))
    tf.logging.info("masked_lm_weights: %s" % " ".join([str(x) for x in masked_lm_weights]))
    tf.logging.info("label: %s (id = %d)" % (example.label, label_id))

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      masked_lm_positions=masked_lm_positions,
      masked_lm_ids=masked_lm_ids,
      masked_lm_weights=masked_lm_weights,
      label_id=label_id,
      is_real_example=True)
  return feature



def file_based_convert_examples_to_features(
    examples, label_list, output_mode, max_seq_length, max_predictions_per_seq, tokenizer, output_file):
  """Convert a set of `InputExample`s to a TFRecord file."""

  writer = tf.python_io.TFRecordWriter(output_file)

  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list, output_mode,
                                     max_seq_length, max_predictions_per_seq, tokenizer)

    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f

    def create_float_feature(values):
      f = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
      return f


    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_int_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    features["masked_lm_positions"] = create_int_feature(feature.masked_lm_positions)
    features["masked_lm_ids"] = create_int_feature(feature.masked_lm_ids)
    features["masked_lm_weights"] = create_float_feature(feature.masked_lm_weights)
    if output_mode == 'classification':
      features["label_ids"] = create_int_feature([feature.label_id])
    elif output_mode == 'regression':
      features["label_ids"] = create_float_feature([feature.label_id])
    else:
      raise KeyError(mode)
    features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(tf_example.SerializeToString())
  writer.close()


def file_based_input_fn_builder(input_file, seq_length, max_predictions_per_seq, is_training,
                                drop_remainder, output_mode):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "masked_lm_positions": tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
      "masked_lm_ids": tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
      "masked_lm_weights": tf.FixedLenFeature([max_predictions_per_seq], tf.float32),
      "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }
  if output_mode == 'classification':
    name_to_features["label_ids"] = tf.FixedLenFeature([], tf.int64)
  elif output_mode == 'regression':
    name_to_features["label_ids"] = tf.FixedLenFeature([], tf.float32)
  else:
    raise KeyError(output_mode)

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      print(name)
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    #batch_size = params["batch_size"]
    batch_size = FLAGS.train_batch_size if is_training else FLAGS.eval_batch_size

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
     
      d = d.repeat()
      d = d.shuffle(buffer_size=100, seed=FLAGS.random_seed)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))
    return d

  return input_fn


def _decode_record(record, name_to_features):
  """Decodes a record to a TensorFlow example."""
  example = tf.parse_single_example(record, name_to_features)

  # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
  # So cast all int64 to int32.
  for name in list(example.keys()):
    t = example[name]
    if t.dtype == tf.int64:
      t = tf.to_int32(t)
    example[name] = t

  return example

def set_rand_seed(seed):
  tf.set_random_seed(seed)

def load_graph():
  # Load adjacent matrix
  nodes_vocab = {}
  with open("%s/nodes_vocab.txt"%(FLAGS.graph_dir),encoding="utf-8") as f:
    for i, line in enumerate(f):
      nodes_vocab.setdefault(line.strip(), i)
  rels_vocab = {}
  with open("%s/relation_vocab.txt"%(FLAGS.graph_dir),encoding="utf-8") as f:
    for i, line in enumerate(f):
      if len(line.split(" ")) == 1:
        rels_vocab.setdefault(line.strip(), i)
      else:
        rels_vocab.setdefault(line.strip().split(" ")[0], int(line.strip().split(" ")[1]))
  np_adj_mat = np.zeros([len(set(rels_vocab.values())), FLAGS.max_nodes, FLAGS.max_nodes], dtype=np.float)
  with open("%s/spellGraphs.txt"%(FLAGS.graph_dir),encoding="utf-8") as f:
    for i, line in enumerate(f):
      e1,e2, rel = line.strip().split("|")
      if rel in rels_vocab:
        np_adj_mat[rels_vocab[rel], nodes_vocab[e1], nodes_vocab[e2]] = 1
        np_adj_mat[rels_vocab[rel], nodes_vocab[e2], nodes_vocab[e1]] = 1

  w2n = []
  vocab = {}
  with open("%s/vocab.txt"%(FLAGS.graph_dir),encoding="utf-8") as f:
    for i, line in enumerate(f):
      word = line.strip()
      vocab.setdefault(word, i)
      if word in nodes_vocab:
        w2n.append(nodes_vocab[word])
      else:
        w2n.append(0)
  n2w = []
  with open("%s/nodes_vocab.txt"%(FLAGS.graph_dir),encoding="utf-8") as f:
    for i, line in enumerate(f):
      word = line.strip()
      if word in vocab:
        n2w.append(vocab[word])
      else:
        n2w.append(0)
  return np_adj_mat, w2n, n2w

def main(_):
  if FLAGS.random_seed is not None:
    set_rand_seed(FLAGS.random_seed)
  tf.logging.set_verbosity(tf.logging.INFO)
  tf.logging.info('config: \n' + tf.flags.FLAGS.flags_into_string())
  

  #if not FLAGS.do_train and not FLAGS.do_eval:
  #  raise ValueError("At least one of `do_train` or `do_eval` must be True.")

  bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

  processor = CSCProcessor()
    # Build train_input_fn
  if FLAGS.do_train:
    train_examples = processor.get_train_full_examples(FLAGS.data_dir)
    num_train_steps = int(
          len(train_examples) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
    tf.gfile.MakeDirs(FLAGS.output_dir)
    print("num_train_steps",num_train_steps)
    #num_warmup_steps = int(0.1 * num_train_steps)
    num_warmup_steps=0
  else:
    num_train_steps = 0
    num_warmup_steps = 0

  input_files, eval_files = [], []
  #for input_pattern in FLAGS.input_file.split(","):
  #  input_files.extend(tf.gfile.Glob(input_pattern))

  #for input_pattern in FLAGS.eval_file.split(","):
  #  eval_files.extend(tf.gfile.Glob(input_pattern))

    

  config = tf.ConfigProto()

  run_config = tf.estimator.RunConfig(
      session_config=config,
      log_step_count_steps=1<<25,
      tf_random_seed=FLAGS.random_seed,
      model_dir=FLAGS.output_dir,
      keep_checkpoint_max=FLAGS.keep_checkpoint_max,
      #save_checkpoints_steps=FLAGS.save_checkpoints_steps if not FLAGS.use_horovod or hvd.rank() == 0 else (1<<25))
      save_checkpoints_steps= num_train_steps / FLAGS.num_train_epochs)
  
  np_adj_mat, w2n, n2w = load_graph()

  model_fn = model_fn_builder(
      #adj_mat = tf.to_int32(tf.convert_to_tensor(np_adj_mat)),
      #w2n = tf.to_int32(tf.convert_to_tensor(w2n)),
      #n2w = tf.to_int32(tf.convert_to_tensor(n2w)),
      adj_mat = np_adj_mat,
      w2n = np.array(w2n),
      n2w = np.array(n2w),
      bert_config=bert_config,
      init_checkpoint=FLAGS.init_checkpoint,
      learning_rate=FLAGS.learning_rate ,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=FLAGS.use_tpu,
      use_one_hot_embeddings=FLAGS.use_tpu)

  estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config)


  if FLAGS.do_train: 
    train_file = os.path.join(FLAGS.output_dir, "train{0}.tf_record".format(FLAGS.max_seq_length))
    label_list = processor.get_labels()
    tokenizer = tokenization.FullTokenizer(
        vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)
    if not os.path.exists(train_file):
      file_based_convert_examples_to_features(
          train_examples, label_list, processor.get_mode(), FLAGS.max_seq_length, FLAGS.max_predictions_per_seq, tokenizer, train_file)
    train_drop_remainder = False
    train_input_fn = file_based_input_fn_builder(
        input_file=train_file,
        seq_length=FLAGS.max_seq_length,
        max_predictions_per_seq=FLAGS.max_predictions_per_seq,
        is_training=True,
        drop_remainder=train_drop_remainder,
        output_mode=processor.get_mode())
    # Build evaluation input_fn
    eval_examples = processor.get_dev_full_examples(FLAGS.data_dir)
    eval_file = os.path.join(FLAGS.output_dir, "eval{0}.tf_record".format(FLAGS.max_seq_length))
    label_list = processor.get_labels()
    tokenizer = tokenization.FullTokenizer(
        vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)
    file_based_convert_examples_to_features(
        eval_examples, label_list, processor.get_mode(), FLAGS.max_seq_length, FLAGS.max_predictions_per_seq, tokenizer, eval_file)
    eval_drop_remainder = False
    eval_input_fn = file_based_input_fn_builder(
        input_file=eval_file,
        seq_length=FLAGS.max_seq_length,
        max_predictions_per_seq=FLAGS.max_predictions_per_seq,
        is_training=False,
        drop_remainder=eval_drop_remainder,
        output_mode=processor.get_mode())



    training_hooks = []
   

    training_hooks.append(LossHook(1000))
    tf.logging.info("***** Running training *****")
    tf.logging.info("  Batch size = %d", FLAGS.train_batch_size)

    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=num_train_steps, hooks=training_hooks)
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, steps=FLAGS.max_eval_steps, start_delay_secs=3600, throttle_secs=(1<<25))
    training.train_and_evaluate(estimator, train_spec, eval_spec)

  elif FLAGS.do_predict:
    test_examples = processor.get_dev_full_examples(FLAGS.data_dir)
    test_file = os.path.join(FLAGS.output_dir, "test{0}.tf_record".format(FLAGS.max_seq_length))
    label_list = processor.get_labels()
    tokenizer = tokenization.FullTokenizer(
        vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)
    file_based_convert_examples_to_features(
        test_examples, label_list, processor.get_mode(), FLAGS.max_seq_length, FLAGS.max_predictions_per_seq, tokenizer, test_file)
    test_drop_remainder = False
    test_input_fn = file_based_input_fn_builder(
        input_file=test_file,
        seq_length=FLAGS.max_seq_length,
        max_predictions_per_seq=FLAGS.max_predictions_per_seq,
        is_training=False,
        drop_remainder=test_drop_remainder,
        output_mode=processor.get_mode())
    print("cyx|")
    result = estimator.predict(input_fn=test_input_fn)
    print("cyx0")
    output_test_file = os.path.join(FLAGS.output_dir, "test_results.txt")
    input_file = os.path.join(FLAGS.output_dir, "input_file.txt")
    vob = {}
    with open(FLAGS.vocab_file, "r",encoding="utf-8") as f:
      for i, line in enumerate(f):
        vob.setdefault(i, line.strip())
    
    log_prob_rows, log_prob_row = [], []
    with tf.gfile.GFile(output_test_file, "w") as writer:
      with tf.gfile.GFile(input_file, "w") as input_writer:
        tf.logging.info("***** Eval results *****")
        for i, res in enumerate(result):
          if i % (FLAGS.max_seq_length) == 0:
            if i != 0:
              writer.write("\n")
              input_writer.write("\n")
              log_prob_rows.append(log_prob_row)
            log_prob_row = []
            input_writer.write("%s\t" % (test_examples[int(i/FLAGS.max_seq_length)].guid))
          log_prob_row.append(res["predictions"])

          #writer.write("%s " % (vob[np.argmax(res["predictions"])]))
          input_writer.write("%s " % (vob[int(res["input_ids"])]))
          if vob[int(res["input_ids"])] not in ['[SEP]','[CLS]','[PAD]']:
            writer.write("%s " % (vob[np.argmax(res["predictions"])]))
          else:
            writer.write("%s " % (vob[int(res["input_ids"])]))
        # Add the last row predictions.
        #log_prob_rows.append(log_prob_row)
    print("cyx1")
    #log_prob_rows = np.array(log_prob_rows)
    #print(log_prob_rows.shape) #[instance_num, max_seq_len, vob_num]

    # Process the output files to the format given by CSC open-source code. 
    # By comparing between input(withError) and Predictions.
    labels_file = os.path.join(FLAGS.output_dir, "label_results.txt")
    with tf.gfile.GFile(labels_file, "w") as labels_writer:
      with tf.gfile.GFile(input_file, "r") as org_file:
        with tf.gfile.GFile(output_test_file, "r") as test_file:
          for k, (pred, inp) in enumerate(zip(test_file, org_file)):
            pid, atl = inp.strip().split("\t")
            output_list = [pid.strip()]
            pred = pred.split(" ")
            atl = atl.split(" ")
            for i, (pt, at) in enumerate(zip(pred[1:], atl[1:])):
              if at == "[SEP]" or at == '[PAD]':
                break
              # Post preprocess with unsupervised methods, 
	      #because unsup BERT always predict punchuation at 1st pos
              if i == 0:
                if pt == "。" or pt == "，":
                  continue
              if pt.startswith("##"):
                 pt = pt.lstrip("##")   
              if at.startswith("##"):
                 at = at.lstrip("##")   
              if pt != at:
                output_list.append(str(i+1))
                output_list.append(pt)
                
            if len(output_list) == 1:
              output_list.append("0")
            labels_writer.write(", ".join(output_list) + "\n") 
    print("cyx2")
    #Compute F1-score
    detect_TP, detect_FP, detect_FN = 0, 0, 0
    correct_TP, correct_FP, correct_FN = 0, 0, 0
    detect_sent_TP, sent_P, sent_N, correct_sent_TP = 0, 0, 0, 0
    dc_TP, dc_FP, dc_FN = 0, 0, 0
    correction_file = os.path.join(FLAGS.output_dir, "correction_results.txt")
    correction_writer = tf.gfile.GFile(correction_file, "w")
    for idx, (pred, actual) in enumerate(zip(open(labels_file,encoding="utf-8"), 
    	open("%s/TestTruthWithError.txt" %FLAGS.data_dir,encoding="utf-8") if FLAGS.with_error else
    	open("%s/TestTruth.txt" %FLAGS.data_dir,encoding="utf-8"))):#遍历每一行 uid, 12, 人, 28, 一
      pred_tokens = pred.strip().split(" ")#uid, /12, /人, /28, /一
      actual_tokens = actual.strip().split(" ")#uid, /12, /人, /28, /衣
      #assert pred_tokens[0] == actual_tokens[0]
      pred_tokens = pred_tokens[1:]#[12, 人, 28, 一]
      actual_tokens = actual_tokens[1:]#[12, 人, 28, 衣]
      detect_actual_tokens = [int(actual_token.strip(",")) \
		for i,actual_token in enumerate(actual_tokens) if i%2 ==0] #[12, 28]
      correct_actual_tokens = [actual_token.strip(",") \
		for i,actual_token in enumerate(actual_tokens) if i%2 ==1] #[人,衣]
      detect_pred_tokens = [int(pred_token.strip(",")) \
		for i,pred_token in enumerate(pred_tokens) if i%2 ==0]#[12, 28]
      _correct_pred_tokens = [pred_token.strip(",") \
		for i,pred_token in enumerate(pred_tokens) if i%2 ==1]#[人,一]

      # Postpreprocess for ACL2019 csc paper which only deal with last detect positions in test data.
      # If we wanna follow the ACL2019 csc paper, we should take the detect_pred_tokens to:

      
      max_detect_pred_tokens = detect_pred_tokens#[12,28]
      
      correct_pred_zip = zip(detect_pred_tokens, _correct_pred_tokens)#[12, 28]  [人,一]
      correct_actual_zip = zip(detect_actual_tokens, correct_actual_tokens)# [12, 28]  [人,衣]
       
      if detect_pred_tokens[0] !=  0:#第一个字符为0为无错误的句子
        sent_P += 1#预测了多少个句子为P
        if sorted(correct_pred_zip) == sorted(correct_actual_zip):
          correct_sent_TP += 1#预测对了多少个句子
      if detect_actual_tokens[0] != 0:#如果该句子真实为错句
        if sorted(detect_actual_tokens) == sorted(detect_pred_tokens): 
          detect_sent_TP += 1#预测正确了多少个句子
        sent_N += 1#一共有多少个句子是错的

   
 
      if detect_actual_tokens[0]!=0:
        detect_TP += len(set(max_detect_pred_tokens) & set(detect_actual_tokens)) #[12, 28] & [12, 28]=2 预测正确的字符个数
        detect_FN += len(set(detect_actual_tokens) - set(max_detect_pred_tokens)) #[12, 28] -[12, 28，29]=0 没预测出来的字符个数
      detect_FP += len(set(max_detect_pred_tokens) - set(detect_actual_tokens)) #[12, 28,29] -[12, 28]=1 预测FP错误的字符数
      
      correct_pred_tokens = []
      #Only check the correct postion's tokens
      for dpt, cpt in zip(detect_pred_tokens, _correct_pred_tokens):
        if dpt in detect_actual_tokens:
          correct_pred_tokens.append((dpt,cpt))



              

      correction_list = [actual.split(" ")[0].strip(",")]
      for dat, cpt in correct_pred_tokens:
        correction_list.append(str(dat))
        correction_list.append(cpt) 
      correction_writer.write(" ,".join(correction_list) + "\n")
       	

      correct_TP += len(set(correct_pred_tokens) & set(zip(detect_actual_tokens,correct_actual_tokens))) 
      correct_FP += len(set(correct_pred_tokens) - set(zip(detect_actual_tokens,correct_actual_tokens)))
      correct_FN += len(set(zip(detect_actual_tokens,correct_actual_tokens)) - set(correct_pred_tokens)) 

      # Caluate the correction level which depend on predictive detection of BERT
      dc_pred_tokens = zip(detect_pred_tokens, _correct_pred_tokens)
      dc_actual_tokens = zip(detect_actual_tokens, correct_actual_tokens)
      dc_TP += len(set(dc_pred_tokens) & set(dc_actual_tokens)) 
      dc_FP += len(set(dc_pred_tokens) - set(dc_actual_tokens)) 
      dc_FN += len(set(dc_actual_tokens) - set(dc_pred_tokens)) 
    
    detect_precision = detect_TP * 1.0 / (detect_TP + detect_FP)
    detect_recall = detect_TP * 1.0 / (detect_TP + detect_FN)
    detect_F1 = 2. * detect_precision * detect_recall/ ((detect_precision + detect_recall) + 1e-8)

    correct_precision = correct_TP * 1.0 / (correct_TP + correct_FP)
    correct_recall = correct_TP * 1.0 / (correct_TP + correct_FN)
    correct_F1 = 2. * correct_precision * correct_recall/ ((correct_precision + correct_recall) + 1e-8)

    dc_precision = dc_TP * 1.0 / (dc_TP + dc_FP + 1e-8)
    dc_recall = dc_TP * 1.0 / (dc_TP + dc_FN + 1e-8)
    dc_F1 = 2. * dc_precision * dc_recall/ (dc_precision + dc_recall + 1e-8)
    if FLAGS.with_error:
      #Token-level metrics
      print("detect_precision=%f, detect_recall=%f, detect_Fscore=%f" %(detect_precision, detect_recall, detect_F1))
      print("correct_precision=%f, correct_recall=%f, correct_Fscore=%f" %(correct_precision, correct_recall, correct_F1))  
      print("dc_joint_precision=%f, dc_joint_recall=%f, dc_joint_Fscore=%f" %(dc_precision, dc_recall, dc_F1))
   
    detect_sent_precision = detect_sent_TP * 1.0 / (sent_P)
    detect_sent_recall = detect_sent_TP * 1.0 / (sent_N)
    detect_sent_F1 = 2. * detect_sent_precision * detect_sent_recall/ ((detect_sent_precision + detect_sent_recall) + 1e-8)

    correct_sent_precision = correct_sent_TP * 1.0 / (sent_P)
    correct_sent_recall = correct_sent_TP * 1.0 / (sent_N)
    correct_sent_F1 = 2. * correct_sent_precision * correct_sent_recall/ ((correct_sent_precision + correct_sent_recall) + 1e-8)

    if not FLAGS.with_error:
      #Sentence-level metrics
      print("detect_sent_precision=%f, detect_sent_recall=%f, detect_Fscore=%f" %(detect_sent_precision, detect_sent_recall, detect_sent_F1))
      print("correct_sent_precision=%f, correct_sent_recall=%f, correct_Fscore=%f" %(correct_sent_precision, correct_sent_recall, correct_sent_F1))  
        
         
        
        
	

if __name__ == "__main__":
  flags.mark_flag_as_required("bert_config_file")
  flags.mark_flag_as_required("output_dir")
  tf.app.run()



INFO:tensorflow:config: 
--bert_config_file=model/bert_config.json
--nocontinual
--data_dir=data/merged/
--nodo_eval
--do_lower_case
--nodo_predict
--do_train
--eval_batch_size=1
--graph_dir=data/gcn_graph.ty_xj
--init_checkpoint=model/bert_model.ckpt
--iterations_per_loop=5000
--job_name=SpellGCN
--keep_checkpoint_max=1
--learning_rate=1.4e-05
--max_eval_steps=10000
--max_nodes=4755
--max_predictions_per_seq=128
--max_relations=2
--max_seq_length=128
--next_sent_type=3
--num_tpu_cores=1
--num_train_epochs=10.0
--num_warmup_steps=10000
--output_dir=output2/
--save_checkpoints_steps=10000
--task_name=CSC
--train_batch_size=32
--nouse_horovod
--nouse_tpu
--vocab_file=model/vocab.txt
--nowith_error
--nohelp
--nohelpfull
--nohelpshort
--nohelpxml
--noonly_check_args
--nopdb_post_mortem
--norun_with_pdb
--norun_with_profiling
--use_cprofile_for_profiling
--noalsologtostderr
--log_dir=
--nologtostderr
--showprefixforinfo
--stderrthreshold=fatal
--verbosity=0
--test_random_seed=301
--test_ra

/usr/local/lib/python3.6/dist-packages/absl/flags/_validators.py:359: UserWarning: Flag --bert_config_file has a non-None default value; therefore, mark_flag_as_required will pass even if flag is not specified in the command line!
  'command line!' % flag_name)
/usr/local/lib/python3.6/dist-packages/absl/flags/_validators.py:359: UserWarning: Flag --output_dir has a non-None default value; therefore, mark_flag_as_required will pass even if flag is not specified in the command line!
  'command line!' % flag_name)
I0812 11:09:48.575912 140698601879424 <ipython-input-3-cf131941b705>:1011] config: 
--bert_config_file=model/bert_config.json
--nocontinual
--data_dir=data/merged/
--nodo_eval
--do_lower_case
--nodo_predict
--do_train
--eval_batch_size=1
--graph_dir=data/gcn_graph.ty_xj
--init_checkpoint=model/bert_model.ckpt
--iterations_per_loop=5000
--job_name=SpellGCN
--keep_checkpoint_max=1
--learning_rate=1.4e-05
--max_eval_steps=10000
--max_nodes=4755
--max_predictions_per_seq=128
--max_

W0812 11:09:48.583466 140698601879424 module_wrapper.py:139] From /content/drive/My Drive/SpellGCN/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



num_train_steps 87931
INFO:tensorflow:Using config: {'_model_dir': 'output2/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 8793.1, '_save_checkpoints_secs': None, '_session_config': , '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 33554432, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff6824e1f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0812 11:10:04.166058 140698601879424 estimator.py:212] Using config: {'_model_dir': 'output2/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 8793.1, '_save_checkpoints_secs': None, '_session_config': , '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 33554432, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff6824e1f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Done running local_init_op.


I0812 11:11:13.727712 140698601879424 session_manager.py:502] Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 0 into output2/model.ckpt.


I0812 11:12:38.931864 140698601879424 basic_session_run_hooks.py:606] Saving checkpoints for 0 into output2/model.ckpt.


ResourceExhaustedError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Saving dict for global step 8794: global_step = 8794, loss = 0.04074172, masked_lm_accuracy = 0.9917238, masked_lm_loss = 0.03608737
#:Saving dict for global step 35176: global_step = 35176, loss = 0.05142015, masked_lm_accuracy = 0.99181277, masked_lm_loss = 0.04511955

In [ ]:
#global_step=9000	total_loss=0.023159	masked_lm_loss=0.023159	learning_rate=1.795294e-05
#global_step=18000	total_loss=0.015242	masked_lm_loss=0.015242	learning_rate=1.590588e-05

In [ ]:
"""\

I0811 05:52:11.224016 140540902344576 basic_session_run_hooks.py:262] loss = 0.7575985, step = 0
INFO:tensorflow:global_step=1000	total_loss=0.073903	masked_lm_loss=0.073903	learning_rate=1.977255e-05
I0811 06:14:25.641248 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=1000	total_loss=0.073903	masked_lm_loss=0.073903	learning_rate=1.977255e-05
INFO:tensorflow:global_step=2000	total_loss=0.037587	masked_lm_loss=0.037587	learning_rate=1.954510e-05
I0811 06:36:06.533686 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=2000	total_loss=0.037587	masked_lm_loss=0.037587	learning_rate=1.954510e-05
INFO:tensorflow:global_step=3000	total_loss=0.061257	masked_lm_loss=0.061257	learning_rate=1.931765e-05
I0811 06:57:47.521117 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=3000	total_loss=0.061257	masked_lm_loss=0.061257	learning_rate=1.931765e-05
INFO:tensorflow:global_step=4000	total_loss=0.074183	masked_lm_loss=0.074183	learning_rate=1.909020e-05
I0811 07:19:27.913834 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=4000	total_loss=0.074183	masked_lm_loss=0.074183	learning_rate=1.909020e-05
INFO:tensorflow:global_step=5000	total_loss=0.025876	masked_lm_loss=0.025876	learning_rate=1.886274e-05
I0811 07:41:08.915226 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=5000	total_loss=0.025876	masked_lm_loss=0.025876	learning_rate=1.886274e-05
INFO:tensorflow:global_step=6000	total_loss=0.052892	masked_lm_loss=0.052892	learning_rate=1.863529e-05
I0811 08:02:50.294044 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=6000	total_loss=0.052892	masked_lm_loss=0.052892	learning_rate=1.863529e-05
INFO:tensorflow:global_step=7000	total_loss=0.023369	masked_lm_loss=0.023369	learning_rate=1.840784e-05
I0811 08:24:32.813528 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=7000	total_loss=0.023369	masked_lm_loss=0.023369	learning_rate=1.840784e-05
INFO:tensorflow:global_step=8000	total_loss=0.029073	masked_lm_loss=0.029073	learning_rate=1.818039e-05
I0811 08:46:13.665036 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=8000	total_loss=0.029073	masked_lm_loss=0.029073	learning_rate=1.818039e-05
INFO:tensorflow:Saving checkpoints for 8794 into output/model.ckpt.
I0811 09:03:25.449968 140540902344576 basic_session_run_hooks.py:606] Saving checkpoints for 8794 into output/model.ckpt.
WARNING:tensorflow:From /tensorflow-1.15.2/python3.6/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
W0811 09:03:30.760559 140540902344576 deprecation.py:323] From /tensorflow-1.15.2/python3.6/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
WARNING:tensorflow:Entity <function file_based_input_fn_builder.<locals>.input_fn.<locals>.<lambda> at 0x7fd1128d42f0> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: module 'gast' has no attribute 'Str'
W0811 09:03:40.111932 140540902344576 ag_logging.py:146] Entity <function file_based_input_fn_builder.<locals>.input_fn.<locals>.<lambda> at 0x7fd1128d42f0> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: module 'gast' has no attribute 'Str'
WARNING: Entity <function file_based_input_fn_builder.<locals>.input_fn.<locals>.<lambda> at 0x7fd1128d42f0> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: module 'gast' has no attribute 'Str'
input_ids
input_mask
is_real_example
label_ids
masked_lm_ids
masked_lm_positions
masked_lm_weights
segment_ids
INFO:tensorflow:Calling model_fn.
I0811 09:03:40.142891 140540902344576 estimator.py:1148] Calling model_fn.
INFO:tensorflow:*** Features ***
I0811 09:03:40.144002 140540902344576 <ipython-input-3-137d0160d200>:387] *** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
I0811 09:03:40.147253 140540902344576 <ipython-input-3-137d0160d200>:389]   name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
I0811 09:03:40.149939 140540902344576 <ipython-input-3-137d0160d200>:389]   name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = is_real_example, shape = (?,)
I0811 09:03:40.152293 140540902344576 <ipython-input-3-137d0160d200>:389]   name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
I0811 09:03:40.154606 140540902344576 <ipython-input-3-137d0160d200>:389]   name = label_ids, shape = (?,)
INFO:tensorflow:  name = masked_lm_ids, shape = (?, 128)
I0811 09:03:40.156091 140540902344576 <ipython-input-3-137d0160d200>:389]   name = masked_lm_ids, shape = (?, 128)
INFO:tensorflow:  name = masked_lm_positions, shape = (?, 128)
I0811 09:03:40.159022 140540902344576 <ipython-input-3-137d0160d200>:389]   name = masked_lm_positions, shape = (?, 128)
INFO:tensorflow:  name = masked_lm_weights, shape = (?, 128)
I0811 09:03:40.161718 140540902344576 <ipython-input-3-137d0160d200>:389]   name = masked_lm_weights, shape = (?, 128)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
I0811 09:03:40.164200 140540902344576 <ipython-input-3-137d0160d200>:389]   name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
I0811 09:03:43.472380 140540902344576 <ipython-input-3-137d0160d200>:442] **** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (21128, 768), *INIT_FROM_CKPT*
I0811 09:03:43.479518 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/embeddings/word_embeddings:0, shape = (21128, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
I0811 09:03:43.484183 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
I0811 09:03:43.491159 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.494143 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.497673 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.500187 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.502162 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.505070 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.508115 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.515135 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.517530 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.519599 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.521682 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.524298 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.526331 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.528348 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.530459 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/dense/kernel:0, shape =  (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.532318 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.534198 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.536031 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.538035 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_0/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.539962 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.541844 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.543832 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.545984 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.548112 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.550345 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.552239 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.553932 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.555711 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.557495 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.559312 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.561029 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.562738 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.564397 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.566096 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.568001 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_1/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.569987 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.571722 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.573392 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.575295 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.577033 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.578737 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.580513 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.582393 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.584263 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.585960 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.587687 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.589391 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.591166 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.593076 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.594766 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.596568 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_2/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.598346 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.600247 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.602259 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.604092 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.605862 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.607939 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.609738 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.611805 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.613787 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.616049 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.618134 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.621106 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.623254 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.625337 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.627167 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.629605 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_3/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.631491 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.633573 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.635307 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.637372 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.639175 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.641000 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.642999 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.644819 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.646822 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.648746 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.650948 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.652881 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.655177 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.657251 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.659642 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.661876 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_4/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.664102 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.666270 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.668503 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.670547 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.672446 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.674577 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.676442 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.678619 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.680489 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.682326 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.684447 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.686230 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.688262 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.690190 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.692001 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.694142 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_5/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.695952 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.697504 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.698977 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.700855 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.702949 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.704748 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.706583 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.708415 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.710273 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.712248 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.714206 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.716123 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.718185 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.720091 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.721884 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.723644 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_6/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.725703 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.727787 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.729647 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.731674 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.733464 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.735240 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.737274 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.739087 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.740978 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.743075 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.745330 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.747622 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.749475 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.751479 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.753249 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.755238 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_7/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.757094 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.759028 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.761088 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.762990 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.764951 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.767134 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.769195 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.771501 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.773451 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.775478 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.777604 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.779595 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.781651 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.783565 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.785412 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.786984 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.788872 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.790740 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.792594 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.794607 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.796447 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.798543 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.800910 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.802731 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.806484 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.808106 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.810600 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.812180 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.813682 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.815261 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.817065 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.819220 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_9/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.821534 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.823520 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.827356 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.830828 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.832660 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.834610 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.836641 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.838622 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.840521 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.842298 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.843992 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.845957 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.847559 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.849328 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.851088 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.852905 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_10/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.854582 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.925593 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.927662 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.929573 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.931367 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.933577 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.935338 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.937122 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.939052 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.940963 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0811 09:03:43.942707 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0811 09:03:43.944642 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0811 09:03:43.946377 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.948157 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.949982 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.951935 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/encoder/layer_11/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/pooler/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.954018 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/pooler/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/pooler/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.955860 140540902344576 <ipython-input-3-137d0160d200>:448]   name = bert/pooler/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = gcn/label-0-name-GCN_layer-0/w_in:0, shape = (768, 768)
I0811 09:03:43.957976 140540902344576 <ipython-input-3-137d0160d200>:448]   name = gcn/label-0-name-GCN_layer-0/w_in:0, shape = (768, 768)
INFO:tensorflow:  name = gcn/label-0-name-GCN_layer-0/b_in:0, shape = (1, 768)
I0811 09:03:43.959951 140540902344576 <ipython-input-3-137d0160d200>:448]   name = gcn/label-0-name-GCN_layer-0/b_in:0, shape = (1, 768)
INFO:tensorflow:  name = gcn/label-1-name-GCN_layer-0/w_in:0, shape = (768, 768)
I0811 09:03:43.962104 140540902344576 <ipython-input-3-137d0160d200>:448]   name = gcn/label-1-name-GCN_layer-0/w_in:0, shape = (768, 768)
INFO:tensorflow:  name = gcn/label-1-name-GCN_layer-0/b_in:0, shape = (1, 768)
I0811 09:03:43.963976 140540902344576 <ipython-input-3-137d0160d200>:448]   name = gcn/label-1-name-GCN_layer-0/b_in:0, shape = (1, 768)
INFO:tensorflow:  name = gcn/relation_prototype_scope/relation_prototype:0, shape = (2, 768, 1)
I0811 09:03:43.965915 140540902344576 <ipython-input-3-137d0160d200>:448]   name = gcn/relation_prototype_scope/relation_prototype:0, shape = (2, 768, 1)
INFO:tensorflow:  name = gcn/label-0-name-GCN_layer-1/w_in:0, shape = (768, 768)
I0811 09:03:43.967801 140540902344576 <ipython-input-3-137d0160d200>:448]   name = gcn/label-0-name-GCN_layer-1/w_in:0, shape = (768, 768)
INFO:tensorflow:  name = gcn/label-0-name-GCN_layer-1/b_in:0, shape = (1, 768)
I0811 09:03:43.969658 140540902344576 <ipython-input-3-137d0160d200>:448]   name = gcn/label-0-name-GCN_layer-1/b_in:0, shape = (1, 768)
INFO:tensorflow:  name = gcn/label-1-name-GCN_layer-1/w_in:0, shape = (768, 768)
I0811 09:03:43.971786 140540902344576 <ipython-input-3-137d0160d200>:448]   name = gcn/label-1-name-GCN_layer-1/w_in:0, shape = (768, 768)
INFO:tensorflow:  name = gcn/label-1-name-GCN_layer-1/b_in:0, shape = (1, 768)
I0811 09:03:43.973609 140540902344576 <ipython-input-3-137d0160d200>:448]   name = gcn/label-1-name-GCN_layer-1/b_in:0, shape = (1, 768)
INFO:tensorflow:  name = cls/predictions/transform/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I0811 09:03:43.975577 140540902344576 <ipython-input-3-137d0160d200>:448]   name = cls/predictions/transform/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = cls/predictions/transform/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.977676 140540902344576 <ipython-input-3-137d0160d200>:448]   name = cls/predictions/transform/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = cls/predictions/transform/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.979852 140540902344576 <ipython-input-3-137d0160d200>:448]   name = cls/predictions/transform/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = cls/predictions/transform/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0811 09:03:43.981784 140540902344576 <ipython-input-3-137d0160d200>:448]   name = cls/predictions/transform/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = cls/predictions/output_bias:0, shape = (21128,), *INIT_FROM_CKPT*
I0811 09:03:43.983633 140540902344576 <ipython-input-3-137d0160d200>:448]   name = cls/predictions/output_bias:0, shape = (21128,), *INIT_FROM_CKPT*
INFO:tensorflow:Done calling model_fn.
I0811 09:03:44.263476 140540902344576 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-11T09:03:44Z
I0811 09:03:44.312300 140540902344576 evaluation.py:255] Starting evaluation at 2020-08-11T09:03:44Z
INFO:tensorflow:Graph was finalized.
I0811 09:03:45.265072 140540902344576 monitored_session.py:240] Graph was finalized.
INFO:tensorflow:Restoring parameters from output/model.ckpt-8794
I0811 09:03:45.503376 140540902344576 saver.py:1284] Restoring parameters from output/model.ckpt-8794
INFO:tensorflow:Running local_init_op.
I0811 09:04:01.266672 140540902344576 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0811 09:04:01.405440 140540902344576 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
I0811 09:08:24.789722 140540902344576 evaluation.py:167] Evaluation [1000/10000]
INFO:tensorflow:Finished evaluation at 2020-08-11-09:08:49
I0811 09:08:49.006539 140540902344576 evaluation.py:275] Finished evaluation at 2020-08-11-09:08:49
INFO:tensorflow:Saving dict for global step 8794: global_step = 8794, loss = 0.04074172, masked_lm_accuracy = 0.9917238, masked_lm_loss = 0.03608737
I0811 09:08:49.008470 140540902344576 estimator.py:2049] Saving dict for global step 8794: global_step = 8794, loss = 0.04074172, masked_lm_accuracy = 0.9917238, masked_lm_loss = 0.03608737
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8794: output/model.ckpt-8794
I0811 09:08:54.107555 140540902344576 estimator.py:2109] Saving 'checkpoint_path' summary for global step 8794: output/model.ckpt-8794
INFO:tensorflow:global_step=9000	total_loss=0.023159	masked_lm_loss=0.023159	learning_rate=1.795294e-05
I0811 09:13:23.677697 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=9000	total_loss=0.023159	masked_lm_loss=0.023159	learning_rate=1.795294e-05
INFO:tensorflow:global_step=10000	total_loss=0.028253	masked_lm_loss=0.028253	learning_rate=1.772549e-05
I0811 09:35:04.017354 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=10000	total_loss=0.028253	masked_lm_loss=0.028253	learning_rate=1.772549e-05
INFO:tensorflow:global_step=11000	total_loss=0.020687	masked_lm_loss=0.020687	learning_rate=1.749804e-05
I0811 09:56:45.212787 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=11000	total_loss=0.020687	masked_lm_loss=0.020687	learning_rate=1.749804e-05
INFO:tensorflow:global_step=12000	total_loss=0.014428	masked_lm_loss=0.014428	learning_rate=1.727059e-05
I0811 10:18:26.869538 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=12000	total_loss=0.014428	masked_lm_loss=0.014428	learning_rate=1.727059e-05
INFO:tensorflow:global_step=13000	total_loss=0.006470	masked_lm_loss=0.006470	learning_rate=1.704314e-05
I0811 10:40:07.681524 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=13000	total_loss=0.006470	masked_lm_loss=0.006470	learning_rate=1.704314e-05
INFO:tensorflow:global_step=14000	total_loss=0.029355	masked_lm_loss=0.029355	learning_rate=1.681568e-05
I0811 11:01:47.416919 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=14000	total_loss=0.029355	masked_lm_loss=0.029355	learning_rate=1.681568e-05
INFO:tensorflow:global_step=15000	total_loss=0.015780	masked_lm_loss=0.015780	learning_rate=1.658823e-05
I0811 11:23:26.437827 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=15000	total_loss=0.015780	masked_lm_loss=0.015780	learning_rate=1.658823e-05
INFO:tensorflow:global_step=16000	total_loss=0.028507	masked_lm_loss=0.028507	learning_rate=1.636078e-05
I0811 11:45:07.254862 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=16000	total_loss=0.028507	masked_lm_loss=0.028507	learning_rate=1.636078e-05
INFO:tensorflow:global_step=17000	total_loss=0.047031	masked_lm_loss=0.047031	learning_rate=1.613333e-05
I0811 12:06:48.532111 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=17000	total_loss=0.047031	masked_lm_loss=0.047031	learning_rate=1.613333e-05
INFO:tensorflow:Saving checkpoints for 17588 into output/model.ckpt.
I0811 12:19:32.534928 140540902344576 basic_session_run_hooks.py:606] Saving checkpoints for 17588 into output/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (33554432 secs).
I0811 12:19:46.209975 140540902344576 training.py:527] Skip the current checkpoint eval due to throttle secs (33554432 secs).
INFO:tensorflow:global_step=18000	total_loss=0.015242	masked_lm_loss=0.015242	learning_rate=1.590588e-05
I0811 12:28:43.973476 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=18000	total_loss=0.015242	masked_lm_loss=0.015242	learning_rate=1.590588e-05
INFO:tensorflow:global_step=19000	total_loss=0.008284	masked_lm_loss=0.008284	learning_rate=1.567843e-05
I0811 12:50:24.144934 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=19000	total_loss=0.008284	masked_lm_loss=0.008284	learning_rate=1.567843e-05
INFO:tensorflow:global_step=20000	total_loss=0.013066	masked_lm_loss=0.013066	learning_rate=1.545098e-05
I0811 13:12:05.268791 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=20000	total_loss=0.013066	masked_lm_loss=0.013066	learning_rate=1.545098e-05
INFO:tensorflow:global_step=21000	total_loss=0.007921	masked_lm_loss=0.007921	learning_rate=1.522353e-05
I0811 13:33:45.184136 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=21000	total_loss=0.007921	masked_lm_loss=0.007921	learning_rate=1.522353e-05
INFO:tensorflow:global_step=22000	total_loss=0.005666	masked_lm_loss=0.005666	learning_rate=1.499608e-05
I0811 13:55:24.133337 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=22000	total_loss=0.005666	masked_lm_loss=0.005666	learning_rate=1.499608e-05
INFO:tensorflow:global_step=23000	total_loss=0.005253	masked_lm_loss=0.005253	learning_rate=1.476863e-05
I0811 14:17:02.831840 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=23000	total_loss=0.005253	masked_lm_loss=0.005253	learning_rate=1.476863e-05
INFO:tensorflow:global_step=24000	total_loss=0.003007	masked_lm_loss=0.003007	learning_rate=1.454117e-05
I0811 14:38:40.474948 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=24000	total_loss=0.003007	masked_lm_loss=0.003007	learning_rate=1.454117e-05
INFO:tensorflow:global_step=25000	total_loss=0.004460	masked_lm_loss=0.004460	learning_rate=1.431372e-05
I0811 15:00:18.697771 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=25000	total_loss=0.004460	masked_lm_loss=0.004460	learning_rate=1.431372e-05
INFO:tensorflow:global_step=26000	total_loss=0.003636	masked_lm_loss=0.003636	learning_rate=1.408627e-05
I0811 15:21:56.479569 140540902344576 <ipython-input-3-137d0160d200>:195] global_step=26000	total_loss=0.003636	masked_lm_loss=0.003636	learning_rate=1.408627e-05
INFO:tensorflow:Saving checkpoints for 26382 into output/model.ckpt.
I0811 15:30:11.298218 140540902344576 basic_session_run_hooks.py:606] Saving checkpoints for 26382 into output/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (33554432 secs).
I0811 15:30:25.938093 140540902344576 training.py:527] Skip the current checkpoint eval due to throttle secs (33554432 secs).
:tensorflow:global_step=35382	total_loss=0.006681	masked_lm_loss=0.006681	learning_rate=8.366628e-06
I0812 04:41:58.034369 139943507625856 <ipython-input-3-6ece556c85da>:195] global_step=35382	total_loss=0.006681	masked_lm_loss=0.006681	learning_rate=8.366628e-06
INFO:tensorflow:global_step=36382	total_loss=0.009452	masked_lm_loss=0.009452	learning_rate=8.207413e-06
I0812 05:03:50.580164 139943507625856 <ipython-input-3-6ece556c85da>:195] global_step=36382	total_loss=0.009452	masked_lm_loss=0.009452	learning_rate=8.207413e-06
INFO:tensorflow:global_step=37382	total_loss=0.000367	masked_lm_loss=0.000367	learning_rate=8.048197e-06
I0812 05:25:41.394645 139943507625856 <ipython-input-3-6ece556c85da>:195] global_step=37382	total_loss=0.000367	masked_lm_loss=0.000367	learning_rate=8.048197e-06
INFO:tensorflow:global_step=38382	total_loss=0.001916	masked_lm_loss=0.001916	learning_rate=7.888981e-06
I0812 05:47:32.905965 139943507625856 <ipython-input-3-6ece556c85da>:195] global_step=38382	total_loss=0.001916	masked_lm_loss=0.001916	learning_rate=7.888981e-06
INFO:tensorflow:global_step=39382	total_loss=0.000450	masked_lm_loss=0.000450	learning_rate=7.729765e-06
I0812 06:09:23.903341 139943507625856 <ipython-input-3-6ece556c85da>:195] global_step=39382	total_loss=0.000450	masked_lm_loss=0.000450	learning_rate=7.729765e-06
INFO:tensorflow:global_step=40382	total_loss=0.001221	masked_lm_loss=0.001221	learning_rate=7.570550e-06
I0812 06:31:16.537547 139943507625856 <ipython-input-3-6ece556c85da>:195] global_step=40382	total_loss=0.001221	masked_lm_loss=0.001221	learning_rate=7.570550e-06
INFO:tensorflow:global_step=41382	total_loss=0.004562	masked_lm_loss=0.004562	learning_rate=7.411334e-06
I0812 06:53:10.013254 139943507625856 <ipython-input-3-6ece556c85da>:195] global_step=41382	total_loss=0.004562	masked_lm_loss=0.004562	learning_rate=7.411334e-06
INFO:tensorflow:global_step=42382	total_loss=0.000755	masked_lm_loss=0.000755	learning_rate=7.252119e-06
I0812 07:15:02.329174 139943507625856 <ipython-input-3-6ece556c85da>:195] global_step=42382	total_loss=0.000755	masked_lm_loss=0.000755	learning_rate=7.252119e-06
INFO:tensorflow:global_step=43382	total_loss=0.003405	masked_lm_loss=0.003405	learning_rate=7.092903e-06
I0812 07:36:56.182432 139943507625856 <ipython-input-3-6ece556c85da>:195] global_step=43382	total_loss=0.003405	masked_lm_loss=0.003405	learning_rate=7.092903e-06
INFO:tensorflow:Saving checkpoints for 43970 into output/model.ckpt.
I0812 07:49:46.209307 139943507625856 basic_session_run_hooks.py:606] Saving checkpoints for 43970 into output/model.ckpt.
"""